#### Lesson Objectives

* learn how to build and visualize a network from spatial data
* learn how to calculate geographic distances
* learn how to extract connectivity properties of a network from the algebraic properties of its graph

In [ ]:
# import common modules
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

#### US state capital dataset

We will use a dataset of the US state capitals which contains their geographic coordinates.

To read the dataset into Python we use the `pandas` built-in function to read csv files.

In [ ]:
# reading the data
data = pd.read_csv('data/state_capitals.csv')

In [ ]:
# data structure
data.head()

Let's directly plot the locations of the cities.

In [ ]:
# plotting the locations of the cities
plt.plot(data['longitude'],data['latitude'],'ro')
plt.ylabel('longitute')
plt.xlabel('latitude')
plt.title('US State Capitals')

#### Calculating distances

Next we want to calculate the distance between every pair of cities. How can we do that?

Clearly, we cannot use the Euclidean distance between the coordinates since the earth is not flat. To calculate a point between any two points on a sphere we can use the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula). (We know that the earth is not a complete spheroid, so this formula is just an approximation.)


So let's create our own distance function:

In [ ]:
import math

def spherical_distance(origin, destination):
    # distance between two points on earth
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # in km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return(d)# in km

In [ ]:
spherical_distance(data[['latitude','longitude']].iloc[0], data[['latitude','longitude']].loc[2])

It is easy to calculate the distance between two cities but it will take a while to calculate the distance for each pair. We can speed up the process by using a useful function in the `scipy` package.

In [ ]:
from scipy.spatial import distance

In [ ]:
# calculate all pairwise distances using a custom function
Dist = distance.pdist(data[['latitude','longitude']],spherical_distance)
Dist.shape

To save space the distance function computes only the upper triangle of a distance matrix. But we can get the full distance matrix:

In [ ]:
Dist = distance.squareform(Dist)

#### Building the city network

We will say that two cities are connected if the distance between them is less than 300km.

The whole city network then can be represented by an *adjacency matrix* A whose element is 1 for nodes which are connected and zero otherwise.

In [ ]:
# Adjacency matrix
A = Dist<300
A.shape

To visualize the network in Python we will use the `networkx` package.

In [ ]:
import networkx as nx

G = nx.from_numpy_matrix(A)

In Python then the network is stored as lists of nodes and edges (which is more efficient than storing the full adjacency matrix):

In [ ]:
G.nodes()

In [ ]:
G.edges()

Let's look at the network.

In [ ]:
nx.draw(G)

Well, that does not look like the original map. 

Let's fix the layout by adding the geospatial coordinates:

In [ ]:
nx.draw(G,np.array(data[['longitude','latitude']]))

In [ ]:
nx.draw(G,np.array(data[['longitude','latitude']]),node_color = 'blue', node_size = 20)

#### Extra properties

Adjacency matrix contains all the information about a labeled graph!

What else can we extract from it?

The *degree* of a node is the number of edges stemming from it.

We realize we can calculate it by summing the values of A along a row/column.

In [ ]:
# degress
d = np.sum(A, axis = 0)
d

The degree matrix can me created as follows:

In [ ]:
# degree matrix
D = np.diag(d)

We can also build the magical *Laplacian matrix*:

In [ ]:
# Laplacian
L = D - A

The algebraic properties of the Laplacian matrix can reveal interesting properties about the network:

* the number of zero eigenvalues of L is equal to the number of connected components
* the value of the second smallest eigenvalue is called algebraic connectivity and describes how connected the graph is overall

Let's compute the eigenvalues:

In [ ]:
from scipy import linalg

In [ ]:
e, v = linalg.eig(L)

In [ ]:
# plot the sorted eigenvalues
plt.plot(np.sort(e),'ro')
plt.title('Eigenvalues of the Laplacian Matrix')

Calculate the number of connected components:

In [ ]:
# connected components = dimension of nullspace of L = dim of L - rank of L
len(L) - np.linalg.matrix_rank(L)

So there are 22 connected components: we can verify this by counting them on the network plot (note the isolated nodes are also considered to be a connected component).

#### Clustering:

We defined the network by setting a threshold on the distance. In general, we can set the weight of the edge between two cities to be equal to the distance and obtain a weighted graph. Similarly, we can build its Laplacian and use its properties to extract clusters in the data.

Tips for large networks:
* use sparse matrices
* avoid direct computations on the matrices by using efficient algorithms (such as Breadth-First-Search)